In [1]:
import numpy as np
import pandas as pd
from sklearn import model_selection, naive_bayes

from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

data = newsgroups.data
target = newsgroups.target

categories = newsgroups.target_names
print(f"Categories: {categories}")

print(f"Category: {categories[target[0]]}")
print(f"Document:\n{data[0]}")

def tokenize(text):
    ttx = text.split()
 
    return ttx

Categories: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
Category: rec.sport.hockey
Document:


I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens ar

In [2]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(data,target,random_state=1)

In [3]:
list_tokens=[]
for i in x_train:
    list_tokens.append(tokenize(i))
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
list_tokens = [[word for word in sublist if word.lower() not in stop_words and word.isalnum()==True] for sublist in list_tokens]

In [4]:
def count_word_frequency(texts):
    word_frequency = {}
    for text in texts:
        for word in text:  
            if word in word_frequency:
                word_frequency[word] += 1
            else:
                word_frequency[word] = 1
    return word_frequency

dic = count_word_frequency(list_tokens)
dic = {k:v for (k,v) in dic.items() if v>80}
dd = list(dic.keys())
len(dd)


2213

In [5]:

list_tokens_test = []
for text in x_test:
    list_tokens_test.append(tokenize(text))

list_tokens_test = [[word for word in sublist if word.lower() not in stop_words and word.isalnum()==True] for sublist in list_tokens_test]


dic_test = count_word_frequency(list_tokens_test)
len(dic_test)


40212

In [6]:
dic_test = {k: v for (k, v) in dic_test.items() if v > 26}
for i in range(11):
    min_key = min(dic_test, key=dic_test.get)
    del dic_test[min_key]
len(dic_test)

2213

In [7]:
X = np.zeros((len(y_train),len(dic)))
XX = np.zeros((len(y_test), len(dic_test)))


In [8]:
dd_test=dd
for i in range(len(x_train)):  
    s = x_train[i] 
    
    for j in range(len(dic)):  
        X[i][j] = s.count(dd[j]) 
        

for i in range(len(x_test)):
    s = x_test[i]
    for j in range(len(dic_test)):
        XX[i][j] = s.count(dd_test[j])


# USING SKLEARN

In [9]:
clf = naive_bayes.MultinomialNB()
clf.fit(X,y_train)

MultinomialNB()

In [10]:
from sklearn.metrics import confusion_matrix as cnf, classification_report as cr
y_pred = clf.predict(XX)
cnf(y_test,y_pred)

array([[119,   1,   1,   0,   0,   1,   1,   3,   7,   7,   0,   4,   1,
          1,   0,  21,   5,   3,   3,  21],
       [  4, 121,   1,  13,  14,  16,   6,   4,   6,   9,   5,  13,  12,
          2,   9,   1,   3,   0,   0,   3],
       [  7,  32,   5,  83,  21,  51,   2,   6,   5,   9,   2,  13,  13,
          3,   1,   3,   1,   0,   5,   1],
       [  3,   6,   0, 143,  49,   0,  14,   6,   0,   7,   6,   2,  19,
          1,   1,   0,   0,   1,   3,   1],
       [  1,   7,   2,  25, 126,   1,  16,   7,   5,   8,   2,   5,  22,
          3,   1,   0,   1,   0,   1,   1],
       [  1,  41,   0,   7,   5, 139,   8,   2,   0,   1,   3,   6,   5,
          3,   3,   1,   1,   0,   2,   2],
       [  0,   4,   1,  11,  11,   1, 160,  18,   6,   3,   6,   7,  20,
          1,   3,   0,   0,   1,   1,   3],
       [  7,   1,   1,   1,   3,   1,  14, 148,  30,  20,   5,   2,   8,
          3,   3,   2,   7,   2,   4,   3],
       [ 11,   0,   2,   3,   0,   0,  12,  16, 162,   9,   1,  

# OWN IMPLEMENTATION

In [11]:
import numpy as np

class NaiveBayes:
    def __init__(self):
        self.classes = None
        self.log_class_priors = None
        self.log_class_word_freqs = None

    def fit(self, X, y):
        self.classes, counts = np.unique(y, return_counts=True)
        self.log_class_priors = np.log(counts) - np.log(np.sum(counts))
        self.log_class_word_freqs = [np.log(X[y==c].sum(axis=0) + 1) - np.log(np.sum(X[y==c]) + X.shape[1]) for c in self.classes]

    def predict(self, X):
        return [self._predict_single_point(x) for x in X]

    def _predict_single_point(self, x):
        log_probs = [self.log_class_priors[i] + np.sum(x * self.log_class_word_freqs[i]) for i in range(len(self.classes))]
        return self.classes[np.argmax(log_probs)]

nb = NaiveBayes()
nb.fit(X, y_train)
y_pred_own = nb.predict(XX)


In [12]:
cnf(y_test,y_pred_own)

array([[119,   1,   1,   0,   0,   1,   1,   3,   7,   7,   0,   4,   1,
          1,   0,  21,   5,   3,   3,  21],
       [  4, 121,   1,  13,  14,  16,   6,   4,   6,   9,   5,  13,  12,
          2,   9,   1,   3,   0,   0,   3],
       [  7,  32,   5,  83,  21,  51,   2,   6,   5,   9,   2,  13,  13,
          3,   1,   3,   1,   0,   5,   1],
       [  3,   6,   0, 143,  49,   0,  14,   6,   0,   7,   6,   2,  19,
          1,   1,   0,   0,   1,   3,   1],
       [  1,   7,   2,  25, 126,   1,  16,   7,   5,   8,   2,   5,  22,
          3,   1,   0,   1,   0,   1,   1],
       [  1,  41,   0,   7,   5, 139,   8,   2,   0,   1,   3,   6,   5,
          3,   3,   1,   1,   0,   2,   2],
       [  0,   4,   1,  11,  11,   1, 160,  18,   6,   3,   6,   7,  20,
          1,   3,   0,   0,   1,   1,   3],
       [  7,   1,   1,   1,   3,   1,  14, 148,  30,  20,   5,   2,   8,
          3,   3,   2,   7,   2,   4,   3],
       [ 11,   0,   2,   3,   0,   0,  12,  16, 162,   9,   1,  